In [1]:
import warnings
warnings.filterwarnings('ignore')

#读取基本包
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

#设置绘图风格
style.use("fivethirtyeight")

#读取线性回归包
import statsmodels.formula.api as smf

In [3]:
# 读取数据
drug_example=pd.DataFrame(dict(sex=["M","M","M","M","M","M","W","W","W","W"],
                              drug=[1,1,1,1,1,0, 1,0,1,0],
                              days=[5,5,5,5,5,8, 2,4,2,4]))
drug_example                       

,sex,drug,days
0,M,1,5
1,M,1,5
2,M,1,5
3,M,1,5
4,M,1,5
5,M,0,8
6,W,1,2
7,W,0,4
8,W,1,2
9,W,0,4


要查看drugs--对住院天数days的影响（应该是负向）

由于是观察数据，不是random的，所以单纯的组间均值差有误差

In [6]:
#计算组间的均值差
diff= drug_example.query("drug==1").days.mean()- drug_example.query("drug==0").days.mean()
diff

-1.1904761904761898

# 子分类估计器计算ATE

子分类估计器，先确定控制变量：性别sex, 按照控制变量分组计算diff, 再加权平均（加权为：控制变量每组的个体数）

In [27]:
# 子分类估计器，先确定控制变量：性别sex, 按照控制变量分组计算diff, 再加权平均（加权为：控制变量每组的个体数）
q1= len(drug_example.query('sex in ["M"]'))
q0= len(drug_example.query('sex in ["W"]'))

diff1=drug_example.query('sex in ["M"]').query("drug==1").days.mean()- drug_example.query('sex in ["M"]').query("drug==0").days.mean()
diff0=drug_example.query('sex in ["W"]').query("drug==1").days.mean()- drug_example.query('sex in ["W"]').query("drug==0").days.mean()

ATE= (q1*diff1 + q0 * diff0)/(q1+q0)
ATE

-2.6

# 回归估计器计算ATE

回归估计器，先确定控制变量：按照控制变量分组，计算各组diff, 再加权平均（加权为：sex控制变量每组的T的方差的相关值）
所以：回归估计器：喜欢高方差的组

In [36]:
# 1. 手动计算回归估计器的ATE
q1= drug_example.query('sex in ["M"]').drug.var()
q0= drug_example.query('sex in ["W"]').drug.var()

diff1=drug_example.query('sex in ["M"]').query("drug==1").days.mean()- drug_example.query('sex in ["M"]').query("drug==0").days.mean()
diff0=drug_example.query('sex in ["W"]').query("drug==1").days.mean()- drug_example.query('sex in ["W"]').query("drug==0").days.mean()

ATE= (q1*diff1 + q0 * diff0)/(q1+q0)
ATE

-2.333333333333333

In [43]:
# 2. 自动计算回归估计器的ATE
model=smf.ols("days~ drug + C(sex)",data=drug_example).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,7.5455,0.188,40.093,0.000,7.100,7.990
C(sex)[T.W],-3.3182,0.176,-18.849,0.000,-3.734,-2.902
drug,-2.4545,0.188,-13.042,0.000,-2.900,-2.010


# 匹配估计器

匹配估计器：将每个干预过的单元与类似未经过干预的单元进行匹配。匹配后计算每个个体的差，再平均。

# 简单的手动匹配

In [46]:
# 读取数据
trainee=pd.read_csv("C:/Users/62678/Desktop/Causal/data/trainees.csv")
trainee.head()

,unit,trainees,age,earnings
0,1,1,28,17700
1,2,1,34,10200
2,3,1,29,14400
3,4,1,25,20800
4,5,1,29,6100


查看 trainee--earning , 控制变量（age）

In [56]:
# 手动匹配一下，将年龄对齐
unique_on_age= (trainee.query("trainees==0").drop_duplicates("age"))
print(unique_on_age.head())  # 先把 trainees==0 挑出来

matches=(trainee.query("trainees==1").merge(unique_on_age,on="age"))

matches = (matches.assign(diff_earnings = lambda d :d.earnings_x -  d.earnings_y))
print(matches.head())

# 计算均值
matches.diff_earnings.mean()

    unit  trainees  age  earnings
19    20         0   43     20900
20    21         0   50     31000
21    22         0   30     21000
22    23         0   27      9300
23    24         0   54     41100
   unit_x  trainees_x  age  earnings_x  unit_y  trainees_y  earnings_y  \
0       1           1   28       17700      27           0        8800   
1      17           1   28       11500      27           0        8800   
2      19           1   28       16300      27           0        8800   
3       2           1   34       10200      34           0       24200   
4       3           1   29       14400      37           0        6200   

   diff_earnings  
0           8900  
1           2700  
2           7500  
3         -14000  
4           8200  


2457.8947368421054

# 涉及到多个控制变量的匹配

In [57]:
#读取数据
med=pd.read_csv("C:/Users/62678/Desktop/Causal/data/medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


mediction -- recovery , 控制变量：sex,age,severity

In [59]:
# 首先初步计算diff
med.query("medication==1").recovery.mean()- med.query("medication==0").recovery.mean()

16.895799546498726

#要用控制变量来匹配，计算两个个体之间的距离，由于各个特征变量的量纲不同，例如性别（0、1）差别大，severity差别小。所以要先归一化。

In [60]:
# 归一化
x= ["sex","age","severity"]
y= "recovery"

for i in x:
    (med[i]-med[i].mean())/med[i].std()    # 每个值减去每一列的均值/每列标准差

med=med.assign(** {i: (med[i]-med[i].mean())/med[i].std() for i in x})  
med.head()

,sex,age,severity,medication,recovery
0,-0.996980,0.280787,1.459800,1,31
1,1.002979,0.865375,1.502164,1,49
2,1.002979,-0.338749,0.057796,0,38
3,1.002979,0.399465,-0.512557,0,35
4,-0.996980,-0.610473,-0.911125,0,15


#匹配：计算两个点之间的距离。（采用：K 近邻算法）

In [70]:
# 先进行匹配

from sklearn.neighbors import KNeighborsRegressor  # sklearn 包中 K近邻算法

treated=med.query("medication==1")
untreated=med.query("medication==0")

mt0= KNeighborsRegressor(n_neighbors=1).fit(untreated[x],untreated[y])   # 储存 未干预的样本(控制变量x--Y 回归)
mt1= KNeighborsRegressor(n_neighbors=1).fit(treated[x],treated[y])       # 储存 被干预的样本（控制变量x--Y 回归)）

predicted= pd.concat([
 # 为 treated的样本，在untreated 样本里面找 =匹配的值
    treated.assign(match= mt0.predict(treated[x])),
 # 为 untreated 的样本， 在 treated 样本寻找匹配值
    untreated.assign(match = mt1.predict(untreated[x]))]
)

predicted.head()

,sex,age,severity,medication,recovery,match
0,-0.996980,0.280787,1.459800,1,31,39.0
1,1.002979,0.865375,1.502164,1,49,52.0
7,-0.996980,1.495134,1.268540,1,38,46.0
10,1.002979,-0.106534,0.545911,1,34,45.0
16,-0.996980,0.043034,1.428732,1,30,39.0


In [71]:
# 匹配估计器计算ATE: 加权均值（若原来是1 ，就是1；若为0，就是-1），所以加全为：2T-1
ATE= np.mean((predicted.recovery-predicted.match)* (2*predicted.medication-1))
ATE

-0.9954

# 匹配偏差

# 修正后的匹配估计器

In [86]:
# 进行匹配

from sklearn.linear_model import LinearRegression  

treated=med.query("medication==1")
untreated=med.query("medication==0")

mt0= KNeighborsRegressor(n_neighbors=1).fit(untreated[x],untreated[y])   # X--Y\ T=0 KNN回归
mt1= KNeighborsRegressor(n_neighbors=1).fit(treated[x],treated[y])       # X--Y\ T=1 KNN回归
# mto.predict(x)--找与X邻近的值所对应的Y
# mt0.kneighbors(x)--找到与X邻近的X的值和index 

# 先 fit 得到 mu_0(x)
ols0= LinearRegression().fit(untreated[x],untreated[y])   # X--Y\ T=0 线性回归
ols1= LinearRegression().fit(treated[x],treated[y])       # X--Y\ T=1 线性回归

# 找到 match了的样本
treated_match_index= mt0.kneighbors(treated[x],n_neighbors=1)[1].ravel()  # treated 里面match了 的untreated的样本的index
untreated_match_index= mt1.kneighbors(untreated[x],n_neighbors=1)[1].ravel()

#
predicted= pd.concat([
 # 为 treated的样本，在untreated 样本里面找 =匹配的值
   (treated.assign(match= mt0.predict(treated[x])).assign(bias_correct=ols0.predict(treated[x])-ols0.predict(untreated.iloc[treated_match_index][x]))),
    (untreated.assign(match = mt1.predict(untreated[x])).assign(bias_correct=ols1.predict(untreated[x])-ols1.predict(treated.iloc[untreated_match_index][x])))
    
])

predicted.head()

,sex,age,severity,medication,recovery,match,bias_correct
0,-0.996980,0.280787,1.459800,1,31,39.0,4.404034
1,1.002979,0.865375,1.502164,1,49,52.0,12.915348
7,-0.996980,1.495134,1.268540,1,38,46.0,1.871428
10,1.002979,-0.106534,0.545911,1,34,45.0,-0.496970
16,-0.996980,0.043034,1.428732,1,30,39.0,2.610159


In [87]:
# 计算修正后的匹配估计器ATE
ATE= np.mean((predicted.recovery-predicted.match-predicted.bias_correct)* (2*predicted.medication-1))
ATE

-7.3626609061413575

# 自动计算匹配估计器

In [91]:
from casualinference import CausalModel

cm=CausalModel(
   Y=med["recovery"].values,
   D=med["medication"].values,
   X=med[["sex","severity","age"]].values
)

cm.est_via_matching(matches=1,bais_adj=True)

print(cm.estimates)

ModuleNotFoundError: No module named 'casualinference'